<a href="https://colab.research.google.com/github/CarlosVaquero/BudgetBuddy/blob/main/nibud_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/CarlosVaquero/BudgetBuddy/


In [26]:
! pip install --upgrade --quiet  html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!pip install langchain_community
!pip install langchain-google-genai
!pip install langchain-chroma
!pip install faiss-cpu
!pip install -U langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:0

In [8]:
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer

from langchain.tools import BaseTool
from pydantic import Field

import requests


import os
import getpass
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display, Markdown
import textwrap
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage, trim_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer

from langchain.tools import BaseTool
from pydantic import Field

import requests


In [6]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ['GOOGLE_API_KEY'] = getpass.getpass('Gemini API Key:')

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")



Gemini API Key:··········
Enter your Tavily API key: ··········


In [7]:

# Create LLM model
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_retries=2
)


In [27]:
class WebsiteContentTool(BaseTool):
    name: str = Field(default="website_content_tool", description="The name of the tool") # Define name as a Pydantic Field with type annotation
    description: str = Field(default="Useful for retrieving the content of a given website URL. Input should be a valid URL.", description="The description of the tool")  # Define description as a Pydantic Field with type annotation

    def _run(self, urls):
        try:
            loader = AsyncHtmlLoader(urls)
            docs = loader.load()
            html2text = Html2TextTransformer()
            docs_transformed = html2text.transform_documents(docs)

            return docs_transformed

        except requests.exceptions.RequestException as e:
            return f"Error fetching URL: {e}"
        except Exception as e:
            return f"An unexpected error occurred: {e}"

    async def _arun(self, url: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("WebsiteContentTool does not support async")


In [33]:
WCT = WebsiteContentTool()

list_of_webpages = ['https://www.nibud.nl/onderwerpen/uitgaven/geld-besparen/',
                    'https://www.nibud.nl/onderwerpen/uitgaven/kosten-energie-water/']

docs_transformed = WCT._run(list_of_webpages)

Fetching pages: 100%|##########| 2/2 [00:00<00:00,  3.40it/s]


In [34]:

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs_transformed)

In [35]:
documents

[Document(metadata={'source': 'https://www.nibud.nl/onderwerpen/uitgaven/geld-besparen/', 'title': 'Geld besparen - Nibud', 'description': 'Je wilt besparen op je uitgaven, maar hoe pak je het aan? Met onze tips en een bespaarplan helpen we je op weg met minder geld uitgeven.', 'language': 'nl-NL'}, page_content='Direct naar content\n\n  * Nieuws\n  * Webwinkel\n\nMenu\n\n  * Onderwerpen\n    * Rondkomen\n    * Sparen\n    * Uitgaven\n    * Geldproblemen\n    * Inkomensondersteuning\n    * Kinderen en jongeren\n    * Wonen\n    * Pensioen\n    * Uit elkaar\n    * Werken\n  * Dossiers\n    * Voldoende inkomen\n    * Financiële weerbaarheid\n    * Betaalbaar wonen\n    * Betaalbare energietransitie\n  * Samenwerken\n    * Voorlichting op maat\n    * Onderzoek en advies\n    * Deskundigheidsbevordering\n    * Cijfers en rekentools\n    * Cursussen, workshops en coaching\n    * Werkvelden\n  * Over het Nibud\n    * Over de organisatie\n    * Onze missie en impact\n    * Financiering\n    *